# LIBRARIES

In [1]:
import pandas as pd
import numpy  as np
import time

from pkg.constants    import *
from pkg.evaluation   import Evaluation
from pkg.utils        import *
from pkg.algorithims  import *
from pkg.neighborhood import *



from pkg.tabu_list import TabuList

from itertools import permutations
from random    import shuffle



# SEED PRODUCTION

In [2]:
# seeds = np.random.randint(0,99999,SEEDS)
# print(seeds)

# Semillas adri
seeds = np.array([12345, 54321, 12221, 22211, 22212])
print(seeds)

[12345 54321 12221 22211 22212]


# EVALUATION OBJECT

In [3]:
evaluation = Evaluation()

# IMPLEMENATION OF ALGORITHIMS

## DATA DUMP

In [4]:
deltas_df   = pd.read_csv(DELTAS_5M_PATH) 
deltas_matrix   = deltas_df.to_numpy()

## GREEDY SEARCH

In [5]:
initial_state = deltas_matrix[0,:]
start_time = time.time()
greedy_sol, greedy_sol_cost = greedy(initial_state, evaluation)
finish_time = time.time()
print(finish_time - start_time)

0.05663776397705078


In [7]:
print(greedy_sol)
print(greedy_sol_cost)

[ 7.  9. 18.  8. 11. 18. 11. 12.  8. 13. 13. 24. 11. 18. 20. 19.]
574.5513854177129


## RANDOM SEARCH

In [8]:
for i in range(SEEDS):
    np.random.seed(seeds[i])
    rs_sol, rs_sol_cost = random_search(evaluation = evaluation)

    print(rs_sol)
    print(rs_sol_cost)


[14. 14. 14. 24. 11.  5.  8. 16. 14. 19. 19. 19. 14.  8.  8. 14.]
437.97332210029685
[ 9. 15. 15. 19. 15. 19. 11. 15. 13. 17. 19. 11.  9.  6. 13. 15.]
451.3225361187038
[14. 12. 14. 18. 12. 16. 18. 12. 18.  4. 18. 18. 12. 10. 14. 16.]
414.6951224947079
[18.  6. 12. 12. 12. 24. 15. 12.  6. 18. 15. 21.  6. 21.  6. 18.]
437.2945702887965
[21. 10. 10. 13. 21.  5.  8. 13.  8. 16. 16. 21. 10. 10. 21. 18.]
437.62379742709754


## LOCAL SEARCH
first best neighbor

In [5]:
granularity = 1

for i in range(SEEDS):

    np.random.seed(seeds[i])

    ls_sol, ls_sol_cost, calls = local_search(
        granularity = granularity,
        evaluation  = evaluation
    )
    
    print(ls_sol, round(ls_sol_cost,4), calls)

  

[18. 12. 12. 14. 20. 14. 14. 12. 10.  8. 12. 25. 15.  2.  6. 25.] 411.7068 919
[12. 12. 20. 14. 19. 14. 13.  8. 11. 14. 16. 24.  2.  7. 22. 12.] 372.4192 2056
[16. 12. 19. 14. 20. 14. 10. 12. 12. 12. 12. 24.  2. 13. 17. 12.] 361.1852 1086
[16. 10. 22. 18. 20. 14. 14. 12.  9.  8. 12. 18.  2. 22.  7. 12.] 411.2291 874
[18. 13. 12. 15. 20. 14. 21. 12. 10.  8. 24. 24.  8.  2.  6. 12.] 417.9571 733


## SIMULATED ANNEALING

In [9]:
# parametros a ajustar:
slots = 1

# k_max: maximo numero iteraciones: en el entorno de 80
# condicion de parada
k_max = 80

# L(T): condicion enfriamiento. Numero fijo para cada iteracion
# Se enfria la temperatura al final de una iteracion
# La iteracion finaliza al visitar todos los vecinos
L = 20


# Esquema de enfriamiento: Cauchy
# tk = t0/(1+k)

# mu: tanto por uno [0,1], indica la probabilidad de aceptar una solucion peor que la inicial
mu = 0.2

# fi: probabilidad de aceptar una solucion un mu por 1 peor que la inicial
fi = 0.2

# Temperatura inicial
t0 = mu * greedy_sol_cost / (- np.log(fi))

# Temperatura final
tf = t0 / (1 + k_max)



In [10]:
for i in range(SEEDS):
    np.random.seed(seeds[i])
    sa_solution, cost_sa = simulated_an(t0,L,tf,slots,evaluation)
    print(sa_solution)
    print(cost_sa)



[12. 10. 20. 17. 19. 14. 13. 17. 11. 26. 17. 25.  2.  6.  6. 13.]
315.27076534163615
[16. 12. 21. 18. 19. 14. 12. 12. 12. 12. 16. 24.  2.  7. 16. 12.]
330.7564742178211
[21. 12. 20. 16. 17. 14.  8. 11. 12. 16. 13. 24.  2. 18.  4. 13.]
362.78228901847797
[18. 12. 19. 21. 20. 13. 10. 10. 12. 13. 16. 25.  3.  7. 14. 14.]
310.4396266696855
[19. 12. 23. 15. 19. 13.  8. 12. 12. 26. 12. 23.  3.  9.  3. 12.]
365.58590769576847


##  TABU SEARCH

In [6]:
# externo
reboots = 4
total_iterations = 40

total_neighbors = 20
tenure = 4

slots = 5 # esto por que????


In [8]:
for seed in seeds:
    np.random.seed(seed)
    ts_sol, ts_sol_cost = tabu_search(
        tenure           = tenure,
        reboots          = reboots,
        total_iterations = total_iterations,
        slots            = slots,
        total_neighbors  = total_neighbors,
        evaluation       = evaluation
    )

    print(ts_sol, round(ts_sol_cost, 4))

iteracion 0, reboot 0
iteracion 1, reboot 0
iteracion 2, reboot 0
iteracion 3, reboot 0
iteracion 4, reboot 0
iteracion 5, reboot 0
iteracion 6, reboot 0
iteracion 7, reboot 0
iteracion 8, reboot 0
iteracion 9, reboot 0
iteracion 0, reboot 1
iteracion 1, reboot 1
iteracion 2, reboot 1
iteracion 3, reboot 1
iteracion 4, reboot 1
iteracion 5, reboot 1
iteracion 6, reboot 1
iteracion 7, reboot 1
iteracion 8, reboot 1
iteracion 9, reboot 1
iteracion 0, reboot 2
iteracion 1, reboot 2
iteracion 2, reboot 2
iteracion 3, reboot 2
iteracion 4, reboot 2
iteracion 5, reboot 2
iteracion 6, reboot 2
iteracion 7, reboot 2
iteracion 8, reboot 2
iteracion 9, reboot 2
iteracion 0, reboot 3
iteracion 1, reboot 3
iteracion 2, reboot 3
iteracion 3, reboot 3
iteracion 4, reboot 3
iteracion 5, reboot 3
iteracion 6, reboot 3
iteracion 7, reboot 3
iteracion 8, reboot 3
iteracion 9, reboot 3
[11.  8. 23. 15. 18. 15. 14. 18. 20. 11. 16. 21. 23.  0.  6. 20.] 361.3094
iteracion 0, reboot 0
iteracion 1, reboot 0
i

In [17]:
def update_frecuency(matrix, vector):

    for i in range(len(vector)):
        matrix[ int(vector[i]) , i ] += 1

    return matrix



def generate_greedy_solution(matrix):

    solution = np.empty(matrix.shape[1])

    # calculamos la inversa
    inversa = 1/matrix

    
    for i in range(matrix.shape[1]):

        sumatorio = inversa[:,i].sum()

        # normalizamos la columna
        inversa[:,i] = inversa[:,i]/sumatorio

        num = np.random.random()
        sum = 0
        control = 0

        for row in range(matrix.shape[0]):
            sum += inversa[row,i]
            if num < sum:
                solution[i] = row
                control = row # indica cuantos slots se han metido
                break
    
        # reajuste de matriz
        if row != 0:
            inversa = inversa[:-row, :]
        if row is matrix.shape[0]-1:
            pass
    
    return solution
        
        

    


# ALGORITHMS ADJUSTMENT